# Analyzing the data

In [1]:
import pandas as pd

df = pd.read_csv("odds datas/test_10_odds_data.csv") # odds_data
df["T_goals"] = df["H_score"] + df["A_score"]
df

,week,HT,HT.1,home,draw,away,over_1,under_1,over_2,under_2,over_3,under_3,over_4,under_4,H_score,A_score,T_goals
0,11,ARS,EVE,1.29,5.52,10.10,1.15,5.34,1.53,2.50,2.45,1.55,4.79,1.18,3,0,3
1,11,BOU,FUL,2.29,3.02,3.53,1.33,3.35,2.07,1.75,3.76,1.27,9.27,1.06,2,0,2
2,11,BUR,CRY,3.02,3.19,2.46,1.27,3.78,1.88,1.92,3.30,1.33,7.58,1.08,1,1,2
3,11,MNC,WOL,1.42,4.55,7.93,1.19,4.74,1.63,2.27,2.69,1.47,5.54,1.14,0,0,0
4,11,SHU,LIV,4.87,3.34,1.83,1.22,4.31,1.74,2.09,2.89,1.42,6.40,1.11,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,23,CHE,SHU,1.46,4.38,7.18,1.21,4.35,1.73,2.11,2.95,1.40,6.30,1.12,1,0,1
886,23,LIV,BRN,1.85,3.47,4.50,1.21,4.44,1.72,2.12,2.86,1.42,6.28,1.12,0,0,0
887,23,ARS,EVE,1.29,5.52,10.10,1.15,5.34,1.53,2.50,2.45,1.55,4.79,1.18,0,1,1
888,23,BUR,FUL,2.43,3.24,3.01,1.33,3.31,2.07,1.76,3.85,1.26,9.22,1.06,1,0,1


In [2]:
check_list = []
odd_list = []
for val_1, val_2 in zip(df["over_3"], df["T_goals"]):
    check_list.append((val_1, val_2))
    if len(check_list) == 10:
        # print(check_list)
        sorted_odds = sorted(check_list, key=lambda x: x[0], reverse=True)
        all_odds = [x[0] for x in sorted_odds]
        for odd in sorted_odds:
            if all_odds.count(odd[0]) == 1:
                odd_list.append(odd)
                break               
        check_list = []

print("Total Stake: ", len(odd_list))
# odd_list

Total Stake:  89


In [3]:
count = 1
martingale_list = []
for x in odd_list:
    if x[1] > 3:
        # print(count)
        martingale_list.append(count)
        count = 1             
    else:
        count += 1

print(martingale_list)
print(sorted(martingale_list, reverse=True)[:50])
print("Hightest Martingale Level:", max(martingale_list))

[3, 1, 19, 1, 3, 4, 1, 20, 1, 2, 1, 4, 1, 2, 2, 1, 5, 4, 1, 2, 1, 1, 3, 3, 1]
[20, 19, 5, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Hightest Martingale Level: 20


In [4]:

bankroll = 100000
loss_amount = 3700
highest_loss = -(loss_amount * 3)
print("Highest Loss :", highest_loss)

single_target_prtg = 1
single_target = (single_target_prtg * bankroll) / 100
print("Single Target -", single_target)

total_target_prtg = 5
total_target = (total_target_prtg * bankroll) / 100
print("Total Target -", total_target)

total_profit = 0
total_loss = 0
for val in martingale_list:
    if val <= 5:
        total_profit += single_target 
        print("WIN :", total_profit)
        if total_profit >= total_target:
            print("---> Target Achieved <---")
    else:
        total_profit -= loss_amount
        print("LOSE :", total_profit)
        if total_profit <= highest_loss:
            print("----------------------------------Stop Loss Reached")

Highest Loss : -11100
Single Target - 1000.0
Total Target - 5000.0
WIN : 1000.0
WIN : 2000.0
LOSE : -1700.0
WIN : -700.0
WIN : 300.0
WIN : 1300.0
WIN : 2300.0
LOSE : -1400.0
WIN : -400.0
WIN : 600.0
WIN : 1600.0
WIN : 2600.0
WIN : 3600.0
WIN : 4600.0
WIN : 5600.0
---> Target Achieved <---
WIN : 6600.0
---> Target Achieved <---
WIN : 7600.0
---> Target Achieved <---
WIN : 8600.0
---> Target Achieved <---
WIN : 9600.0
---> Target Achieved <---
WIN : 10600.0
---> Target Achieved <---
WIN : 11600.0
---> Target Achieved <---
WIN : 12600.0
---> Target Achieved <---
WIN : 13600.0
---> Target Achieved <---
WIN : 14600.0
---> Target Achieved <---
WIN : 15600.0
---> Target Achieved <---


In [5]:
win_length = sorted(martingale_list, reverse=True)
total_diff = 0
for x in win_length:
    if x > 5:
        total_diff += 1
    else:
        break
print("Losses:", total_diff)
print("Wins:", len(martingale_list) - total_diff)

Losses: 2
Wins: 23
